https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset

In [53]:
import pandas as pd
import numpy as np  


aisles = pd.read_csv(r'kaggle/archive/aisles.csv')
departments = pd.read_csv(r'kaggle/archive/departments.csv')
prior = pd.read_csv(r'kaggle/archive/order_products__prior.csv')
train = pd.read_csv(r'kaggle/archive/order_products__train.csv')
orders = pd.read_csv(r'kaggle/archive/orders.csv')
products = pd.read_csv(r'kaggle/archive/products.csv')

print(aisles.head())
print(departments.head())
print(prior.head())
print(train.head())
print(orders.head())
print(products.head())
products = pd.merge(products, aisles, on='aisle_id')
products = pd.merge(products, departments, on='department_id')

print(prior.shape)
print(train.shape)
print(orders.shape)


   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          

In [54]:
prior = pd.merge(prior,products, on='product_id', how='left')
train = pd.merge(train,products, on='product_id', how='left')
prior = prior[['order_id', 'product_id', 'aisle_id', 'department_id']]
train = train[['order_id', 'product_id', 'aisle_id', 'department_id']]
prior = prior.groupby('order_id').agg({
'product_id': list,
'aisle_id': list,
'department_id': list
}).reset_index()
train = train.groupby('order_id').agg({
'product_id': list,
'aisle_id': list,
'department_id': list
}).reset_index()
orders = orders[orders['eval_set'] != 'test'] # this is data in kaggle to put into model to predict
orders = orders[['order_id', 'user_id', 'days_since_prior_order','order_number']]
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)
orders = pd.merge(orders, prior, on='order_id', how='left')
orders = pd.merge(orders, train, on='order_id', how='left')

print(orders.head())

   order_id  user_id  days_since_prior_order  order_number  \
0   2539329        1                     0.0             1   
1   2398795        1                    15.0             2   
2    473747        1                    21.0             3   
3   2254736        1                    29.0             4   
4    431534        1                    28.0             5   

                                        product_id_x  \
0                  [196, 14084, 12427, 26088, 26405]   
1           [196, 10258, 12427, 13176, 26088, 13032]   
2                  [196, 12427, 10258, 25133, 30450]   
3                  [196, 12427, 10258, 25133, 26405]   
4  [196, 12427, 10258, 25133, 10326, 17122, 41787...   

                          aisle_id_x              department_id_x  \
0               [77, 91, 23, 23, 54]          [7, 16, 19, 19, 17]   
1         [77, 117, 23, 24, 23, 121]       [7, 19, 19, 4, 19, 14]   
2              [77, 23, 117, 21, 88]          [7, 19, 19, 16, 13]   
3             

In [55]:
orders['product_id_x'] = orders['product_id_x'].fillna(orders['product_id_y'])
orders['aisle_id_x'] = orders['aisle_id_x'].fillna(orders['aisle_id_y'])
orders['department_id_x'] = orders['department_id_x'].fillna(orders['department_id_y'])
orders.rename(columns={'product_id_x': 'product_id'}, inplace=True)
orders.rename(columns={'aisle_id_x': 'aisle_id'}, inplace=True)
orders.drop('product_id_y', axis=1, inplace=True)
orders.drop(['aisle_id_y', 'department_id_y'], axis=1, inplace=True)
# thinking about whether to keep department_id or not
orders.drop('department_id_x', axis=1, inplace=True)

print(orders.iloc[0,:])
print(orders.shape)
print(orders.columns)

order_id                                            2539329
user_id                                                   1
days_since_prior_order                                  0.0
order_number                                              1
product_id                [196, 14084, 12427, 26088, 26405]
aisle_id                               [77, 91, 23, 23, 54]
Name: 0, dtype: object
(3346083, 6)
Index(['order_id', 'user_id', 'days_since_prior_order', 'order_number',
       'product_id', 'aisle_id'],
      dtype='object')


# Sort values and then drop column

In [56]:
orders = orders.sort_values(by = ['user_id', 'order_number'],ascending=[True,True])
orders.drop('order_number', axis=1, inplace=True)

# Save to file

In [57]:
orders.to_csv('../ml/data/kaggle_prepared.csv', index=False)